In [1]:
#!pip install matplotlib
#!pip install tensorflow


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


#  Preparing Data

In [17]:
import os
# Directory to check for existence
train_dir = 'train'
# Directory to check for existence
valid_dir = 'valid'



In [18]:
height,width=180,180



In [19]:
## Augment the data
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.2),
    tf.keras.layers.experimental.preprocessing.RandomTranslation(
        height_factor=0.2, 
        width_factor=0.2, 
        fill_mode='reflect'
    ),
])


In [24]:
#train_set = tf.keras.preprocessing.image_dataset_from_directory(train_dir,image_size=(height,width))
#valid_set = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,image_size=(height,width))



train_set = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(height, width),
    shuffle=True,
)
class_names = train_set.class_names
#train_set = train_set.map(lambda x, y: (data_augmentation(x, training=True), y))

valid_set = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir,
    image_size=(height, width),
)

batches = 20
train_set = train_set.take(batches)
valid_set = valid_set.take(batches)


Found 81946 files belonging to 10 classes.
Found 10244 files belonging to 10 classes.


In [10]:
#train_set.class_names, valid_set.class_names # ensure the label in here are equal
class_names, valid_set.class_names


AttributeError: '_TakeDataset' object has no attribute 'class_names'

In [11]:
train_set

<_TakeDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [9]:
valid_set = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,image_size=(height,width))

Found 10244 files belonging to 10 classes.


In [12]:
valid_set

<_TakeDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [10]:
from tensorflow.python.keras.layers import Dense, Flatten


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

# Define your model architecture
model = Sequential([
    # Add layers according to your requirement
    Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # Adjust the number of units to match the number of classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_set, validation_data=valid_set, epochs=10)
model.save_weights('sequential.h5')



Epoch 1/10
 111/2561 [>.............................] - ETA: 7:47 - loss: 1163.1349 - accuracy: 0.1990

In [25]:
dnn_model = Sequential()

imported_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(180, 180, 3),
    pooling='avg',
    classes=10,
    weights='imagenet')

# Freeze all layers in the imported model
for layer in imported_model.layers:
    layer.trainable = False

# Check if the layers are trainable
for layer in imported_model.layers:
    print(f"Layer {layer.name}: Trainable = {layer.trainable}")


dnn_model.add(imported_model)
dnn_model.add(Flatten())
dnn_model.add(Dense(600, activation='relu'))
dnn_model.add(Dense(10, activation='softmax'))



#from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
dnn_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])


Layer input_3: Trainable = False
Layer conv1_pad: Trainable = False
Layer conv1_conv: Trainable = False
Layer conv1_bn: Trainable = False
Layer conv1_relu: Trainable = False
Layer pool1_pad: Trainable = False
Layer pool1_pool: Trainable = False
Layer conv2_block1_1_conv: Trainable = False
Layer conv2_block1_1_bn: Trainable = False
Layer conv2_block1_1_relu: Trainable = False
Layer conv2_block1_2_conv: Trainable = False
Layer conv2_block1_2_bn: Trainable = False
Layer conv2_block1_2_relu: Trainable = False
Layer conv2_block1_0_conv: Trainable = False
Layer conv2_block1_3_conv: Trainable = False
Layer conv2_block1_0_bn: Trainable = False
Layer conv2_block1_3_bn: Trainable = False
Layer conv2_block1_add: Trainable = False
Layer conv2_block1_out: Trainable = False
Layer conv2_block2_1_conv: Trainable = False
Layer conv2_block2_1_bn: Trainable = False
Layer conv2_block2_1_relu: Trainable = False
Layer conv2_block2_2_conv: Trainable = False
Layer conv2_block2_2_bn: Trainable = False
Layer co

# Training The Model 

In [26]:
history = dnn_model.fit(
train_set,
validation_data=valid_set,
epochs=10
)

Epoch 1/10


20/20 [==============================] - 29s 1s/step - loss: 1.4727 - accuracy: 0.5562 - val_loss: 0.6949 - val_accuracy: 0.7656
Epoch 2/10
20/20 [==============================] - 29s 1s/step - loss: 0.3330 - accuracy: 0.8750 - val_loss: 0.4455 - val_accuracy: 0.8594
Epoch 3/10
20/20 [==============================] - 28s 1s/step - loss: 0.1927 - accuracy: 0.9344 - val_loss: 0.3571 - val_accuracy: 0.8875
Epoch 4/10
20/20 [==============================] - 29s 1s/step - loss: 0.0981 - accuracy: 0.9750 - val_loss: 0.3730 - val_accuracy: 0.8750
Epoch 5/10
20/20 [==============================] - 29s 1s/step - loss: 0.0767 - accuracy: 0.9812 - val_loss: 0.3497 - val_accuracy: 0.8891
Epoch 6/10
20/20 [==============================] - 31s 2s/step - loss: 0.0433 - accuracy: 0.9953 - val_loss: 0.4157 - val_accuracy: 0.8859
Epoch 7/10
20/20 [==============================] - 29s 1s/step - loss: 0.0259 - accuracy: 1.0000 - val_loss: 0.3983 - val_accuracy: 0.8750
Epoch 8/10
20/20 [=============

In [27]:
dnn_model.save_weights('resnet_no_augmentation_batch20.h5')


In [16]:
history = dnn_model.fit(
train_set,
validation_data=valid_set,
epochs=15
)

Epoch 1/15
5/5 [==============================] - 8s 2s/step - loss: 0.2942 - accuracy: 0.9000 - val_loss: 0.7538 - val_accuracy: 0.7688
Epoch 2/15
5/5 [==============================] - 8s 2s/step - loss: 0.2508 - accuracy: 0.9125 - val_loss: 0.8231 - val_accuracy: 0.7500
Epoch 3/15
5/5 [==============================] - 8s 2s/step - loss: 0.2690 - accuracy: 0.8938 - val_loss: 0.7567 - val_accuracy: 0.7563
Epoch 4/15
5/5 [==============================] - 8s 2s/step - loss: 0.2930 - accuracy: 0.8875 - val_loss: 0.8769 - val_accuracy: 0.7688
Epoch 5/15
5/5 [==============================] - 8s 2s/step - loss: 0.4086 - accuracy: 0.8687 - val_loss: 0.8456 - val_accuracy: 0.7125
Epoch 6/15
5/5 [==============================] - 8s 2s/step - loss: 0.3574 - accuracy: 0.8938 - val_loss: 0.5972 - val_accuracy: 0.7812
Epoch 7/15
5/5 [==============================] - 8s 2s/step - loss: 0.2987 - accuracy: 0.9062 - val_loss: 0.8477 - val_accuracy: 0.7812
Epoch 8/15
5/5 [=========================